In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic

In [3]:
import numpy as np

a = np.array([32.49, -39.96,-3.86])
b = np.array([31.39, -39.28, -4.66])
c = np.array([31.14, -38.09,-4.49])

def calc_angle(a, b, c):

    a = np.array([a.x, a.y, a.z])
    b = np.array([b.x, b.y, b.z])
    c = np.array([c.x, c.y, c.z])


    ba = a - b
    bc = c - b

    print("Ba:" + str(ba))
    print("BC:" + str(bc))

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)

def get_dist(a, b):
    return np.linalg.norm(np.array([b.x, b.y, b.z]) - np.array([a.x, a.y, a.z]))

In [ ]:
import math
import traceback

cap = cv2.VideoCapture(0)

# Set up OpenCV window
# Changed the window title to 'Deadlift Form Corrector'
cv2.namedWindow('Deadlift Form Corrector', cv2.WINDOW_NORMAL)
# Resized the window to 1200x600 pixels for a wider view
cv2.resizeWindow('Deadlift Form Corrector', 1200, 600)

# Curl counter variables
counter = 0
stage = None

alr_ran = False
og_l_should_pos = 0
og_r_should_pos = 0

state = "none"
reps = 0



# Setup mediapipe instance
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic, mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
        results_face = holistic.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        stage = []
        # Extract landmarks
        try:
            #print("Testing")
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            
            #print(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])
            hip_dist = get_dist(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value], landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
            ankle_dist = get_dist(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value], landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])

            wrist_dist = get_dist(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value], landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
            shoulder_dist = get_dist(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])
            
            l_shoulder_hip_dist = get_dist(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
            r_shoulder_hip_dist = get_dist(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])


            right_elbow_angle = calc_angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value], 
                                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
            left_elbow_angle = calc_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value], 
                                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])

            right_knee_angle = calc_angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value], landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value], 
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])

            left_knee_angle = calc_angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value], landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value], 
                                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])


            new_state = ""
            if left_knee_angle >= 120 or right_knee_angle >= 120:
                new_state = "up"

            if left_knee_angle <= 80 or right_knee_angle <= 80:
                new_state = "down"
            
            if state == "up" and new_state == "down":
                reps += 1
            
            state = new_state


            #print("Right knee:" + str(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]))
            #print("Right hip:" + str(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]))

            # cv2.putText(image, "Hip dis:" + str(hip_dis), 
            #                 (200,200), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            # cv2.putText(image, "Left Elbow Angle:" + str(round(left_elbow_angle, 4)), 
            #                 (300,300), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            # cv2.putText(image, "Left Knee Angle:" + str(round(left_knee_angle, 4)), 
            #                 (400,400), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
            

            #These are the arm angles to see if the user is spreading/moving their arms too much
            # cv2.putText(image, str(l_arm_angle), 
            #                 tuple(np.multiply(l_shoulder, [640, 480]).astype(int)), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            # cv2.putText(image, str(r_arm_angle), 
            #                 tuple(np.multiply(r_shoulder, [640, 480]).astype(int)), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                      )

            
            # Checking Logic
            
            if hip_dist / ankle_dist > 0.65:
                stage.append("Your legs are too narrow")
            if hip_dist / ankle_dist < 0.4:
                stage.append("Your legs are too wide") 
            if shoulder_dist / wrist_dist > 1:
                stage.append("Move your arms farther apart on the bar")
            if shoulder_dist / wrist_dist < 0.58:
                stage.append("Move your arms close together on the bar")  

            if left_elbow_angle < 120:
                stage.append("Keep your arms straight, it can strain your elbows")
            
            if left_knee_angle < 50 or right_knee_angle < 50:
                stage.append("Don't bend down too much, it can strain your knees")
            
            if len(stage) == 0:
                    stage.append("Great Form!")# Render video frame to the left


                       
        except Exception:
            traceback.print_exc()
        
        # Render curl counter

# Setup s# Render video frame to the left
        # Changed the color of the status box to blue
        #cv2.rectangle(image, (0, 0), (720, 130), (0, 0, 255), -1)

        # Render project title
        cv2.putText(image, 'MotionMentor Deadlift Form Corrector', (540, 40),  # Centered at the top
                    cv2.FONT_HERSHEY_DUPLEX, 1.3, (255, 255, 255), 2, cv2.LINE_AA)

        # Render blue box with text at the top
        #cv2.rectangle(image, (0, 0), (1200, 60), (70, 130, 180), -1)  # Blue box
        # Stage data
        # cv2.putText(image, 'MOTION MENTOR', (550,30), 
        #             cv2.FONT_HERSHEY_TRIPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
        
        # cv2.putText(image, "Reps: " + str(reps), 
        #                     (1000,100), 
        #                     cv2.FONT_HERSHEY_DUPLEX, 1.8, (255,255,255), 2, cv2.LINE_AA)

        if(len(stage) == 0):
            continue
        elif stage[0] == "Great Form!":
            cv2.putText(image, stage[0], 
                            (30,100), 
                            cv2.FONT_HERSHEY_DUPLEX, 1.8, (0,255,0), 2, cv2.LINE_AA)
        else:
            i = 0
            for x in stage:
                cv2.putText(image, x,
                            (30,120 + i*60), 
                            cv2.FONT_HERSHEY_DUPLEX, 1.5, (0,0,255), 2, cv2.LINE_AA)
                i += 1

        print(stage)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results_face.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2) 
                                 )               
                                 
        
        cv2.imshow('Deadlift Form Corrector', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
        